# Dynamics via acceleration maps

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from analysis import analysis

mu_factor = 6   # mu
min_factor = 8  # min

#### Load

In [ ]:
global_pos = pd.read_pickle("position_D_0.05.pkl")
global_pos.rename(columns={"grid id" : "gid", "run id" : "rid"}, inplace=True)
print(global_pos.D.unique(), global_pos.R_eq.unique())

global_pos.x *= mu_factor
global_pos.y *= mu_factor

global_pos

#### Position distribution

In [ ]:
analysis.view_position_dist(global_pos, mu_factor)

# escaped_gids = global_pos.query("y > 170 or y < 130 or x > 250 or x < 50").gid.to_list()
# escaped_runs = global_pos.query("y > 170 or y < 130 or x > 250 or x < 50").rid.to_list()

# print(f"Eliminating {len(escaped_gids)} runs..")
# global_pos = global_pos.query("gid != @escaped_gids and rid != @escaped_runs")
# analysis.view_position_dist(global_pos, mu_factor)


### Hopping times

In [ ]:
hop_times = []
for gid, df_gid in global_pos.groupby("gid"):
    for rid, df in df_gid.groupby("rid"):
        t = analysis.get_hopping_times(df, mu_factor, min_factor)
        [hop_times.append(el) for el in t]


In [ ]:
plt.figure(figsize=(3, 3))
plt.hist(np.array(hop_times)/60, bins=50, color="red", alpha=0.5)
plt.xlabel("Hopping time (hr)")
plt.ylabel("Counts")
plt.show()

### Acceleration maps

In [ ]:
global_x_v_a = []

for gid, df_gid in global_pos.groupby("gid"):
    for rid, df in df_gid.groupby("rid"):
        x_v_a = analysis.calc_v_a_from_position(df.x, dt=0.002 * 500 * min_factor)
        x_v_a["gid"] = gid
        x_v_a["rid"] = rid
        global_x_v_a.append(x_v_a)

global_x_v_a = pd.concat(global_x_v_a)
global_x_v_a


In [ ]:
bounds = global_x_v_a.agg(["min", max])
xmin, xmax = bounds["x"]
vmin, vmax = bounds["v"]

nbins = 30
dx = (xmax - xmin) / nbins
dv = (vmax - vmin) / nbins

print(xmin, xmax)
print(vmin, vmax)

In [ ]:
x1 = ("x", xmin, dx)
x2 = ("v", vmin, dv)

analysis.get_bin_indices(global_x_v_a, nbins, x1, x2)
global_x_v_a

In [ ]:
map_a = analysis.map_a(global_x_v_a, nbins, "x_bin", "v_bin")

from matplotlib.ticker import FormatStrFormatter

plt.imshow(map_a, origin="lower", interpolation="none", cmap="rainbow")
plt.xticks(
    [0, nbins // 2, nbins],
    [round(xmin, 2), round((xmin + xmax) / 2, 2), round(xmax, 2)],
)
plt.yticks(
    [0, nbins // 2, nbins], [round(vmin, 2), round((vmin + vmax) / 2, 0), round(vmax, 2)]
)
cbar = plt.colorbar()
cbar.set_label(r"$F$ ($\mu$m/hr$^2$)")
plt.xlabel(r"$x$ ($\mu$m)")
plt.ylabel(r"$v$ ($\mu$m/hr)")
plt.show()


### Streamlines

In [ ]:
X, Y = np.meshgrid(np.linspace(xmin, xmax, nbins), np.linspace(vmin, vmax, nbins))

V = np.empty(X.shape)
V[:] = np.nan

for (j, i), df in global_x_v_a.groupby(["x_bin", "v_bin"]):
    a = df.a.mean()
    V[i, j] = a 

plt.streamplot(X, Y, Y, V, linewidth=2, color=V, cmap="rainbow")
plt.xlabel(r"$x$ ($\mu$m)")
plt.ylabel(r"$v$ ($\mu$m/hr)")
plt.show()

In [ ]:
c = plt.get_cmap("Set2")(3)
plt.quiver(X, Y, Y, V, np.where(V>0, 1, 0), cmap="Set2", width=0.007)
plt.streamplot(X, Y, Y, V, linewidth=2, color=c)
plt.xlabel(r"$x$ ($\mu$m)")
plt.ylabel(r"$v$ ($\mu$m/hr)")
plt.show()